In [ ]:
# gerar_dataset_paralelo_simples.py
import csv
import os
import random
import time
import numpy as np
from collections import defaultdict
import copy
import multiprocessing
import math
from montecarlo import MonteCarlo_Player, MonteCarloNode
from decisiontree import *
from board import Board
from variables import PLAYER1, PLAYER2, COLS, ROWS

def get_valid_moves(board):
    return board.get_legal_moves()

def flatten_board(board):
    flat = []
    grid = board.get_board()
    for r in range(ROWS):
        for c in range(COLS):
            flat.append(grid[r][c])
    return flat

def jogador_aleatorio(board):
    valid_moves = get_valid_moves(board)
    for move in valid_moves:
        temp = board.clone()
        jogador = PLAYER2 if board.get_current_player() == PLAYER1 else PLAYER1
        temp.player = jogador
        temp.drop_piece(move)
        if temp.check_win(jogador):
            return move
    return random.choice(valid_moves) if valid_moves else -1


def simular_jogo_e_coletar_dados(args):
    """
    Simula UM jogo completo e retorna uma lista de linhas de dados (estado, melhor_jogada).
    Recebe as *configurações* (strings de dificuldade) e instancia os jogadores aqui.
    """
    modo_jogo, p1_setting, p2_setting, game_id = args
    board = Board()
    dados_jogo = [] # Armazena as linhas (estado, jogada) deste jogo
    print(f"[Game {game_id}] Iniciando jogo: {modo_jogo} (P1='{p1_setting}', P2='{p2_setting}')")
    jogador1 = None
    jogador2 = None
    try:
        if modo_jogo == 'mcts_vs_random':
            jogador1 = MonteCarlo_Player(difficulty=p1_setting) # Cria instância aqui
            jogador2 = jogador_aleatorio
        elif modo_jogo == 'mcts_vs_mcts':
            if 'dt' in p1_setting:
                if p1_setting == 'dt_easy':
                    jogador1 = DecisionTree_Player(random=False, difficulty='easy')
                elif p1_setting == 'dt_medium': 
                    jogador1 = DecisionTree_Player(random=False, difficulty='medium') # Cria instância aqui
                elif p1_setting == 'dt_hard':
                    jogador1 = DecisionTree_Player(random=False, difficulty='hard')
            else:
                jogador1 = MonteCarlo_Player(difficulty=p1_setting) # Cria instância aqui
            if 'dt' in p2_setting:
                if p2_setting == 'dt_easy':
                    jogador2 = DecisionTree_Player(random=False, difficulty='easy') # Cria instância aqui
                elif p2_setting == 'dt_medium':
                    jogador2 = DecisionTree_Player(random=False, difficulty='medium')
                elif p2_setting == 'dt_hard':
                    jogador2 = DecisionTree_Player(random=False, difficulty='hard')
            else:
                jogador2 = MonteCarlo_Player(difficulty=p2_setting) # Cria instância aqui
        elif modo_jogo == 'random_vs_mcts':
            jogador1 = jogador_aleatorio
            jogador2 = MonteCarlo_Player(difficulty=p2_setting) # Cria instância aqui
        else:
            raise ValueError(f"Modo de jogo inválido: {modo_jogo}")
    except Exception as e:
        print(f"[Game {game_id}] erro ao criar jogadores com settings P1='{p1_setting}', P2='{p2_setting}': {e}")
        print("Verifique se as strings de dificuldade são válidas em MonteCarlo_Player.__init__")
        return [] # Retorna lista vazia se não puder criar jogadores

    # Loop do Jogo
    while not board.is_game_over():
        current_player_id = board.get_current_player()
        current_player_obj = jogador1 if current_player_id == PLAYER1 else jogador2
        move = -1

        valid_moves = get_valid_moves(board)
        if not valid_moves: break

        is_mcts_player = isinstance(current_player_obj, MonteCarlo_Player) or isinstance(current_player_obj, DecisionTree_Player)

        if is_mcts_player:
            try:
                if isinstance(current_player_obj, DecisionTree_Player):
                    best_move_col = current_player_obj.play(state=flatten_board(board.clone()), legal_moves=valid_moves)
                else:
                    best_move_col = current_player_obj.make_move(board.clone())
                    
                move = best_move_col
                if (move not in valid_moves):
                    break

            except Exception as e:
                print(f"[Game {game_id}] ERRO durante MCTS make_move (J{current_player_id}, diff='{current_player_obj.difficulty}'): {e}")
                if valid_moves: move = random.choice(valid_moves) # Tenta jogada aleatória se MCTS falhou
                else: break # Sem jogadas, termina

        else:
            move = current_player_obj(board)
            print(f"[random] Jogada aleatória: {move}")

        # Aplicar a jogada
        if move in valid_moves:
            board.drop_piece(move)
        elif move != -1 :
             break
        else:
             break # Nenhuma jogada definida
        if (isinstance(current_player_obj, MonteCarlo_Player)):
            time.sleep(5)

    
    time.sleep(3)
    return (board.get_winner(), p1_setting, p2_setting)


def main():
    NUM_GAMES_ESTA_EXECUCAO = 1200 # Jogos a gerar nesta execução

    # Define os matchups *usando apenas as strings de dificuldade* que MonteCarlo_Player entende
    DESIRED_MATCHUPS = [
        ('easy', 'dt_hard'),
        ('medium', 'dt_hard'),
        ('hard', 'dt_hard'), 
        ('dt_hard', 'easy'),
        ('dt_hard', 'medium'),
        ('dt_hard', 'hard'),
    ]
    MODE = 'mcts_vs_mcts' # Deve ser consistente com os matchups

    FILENAME = 'dataset_quatro_em_linha_mcts.csv' # Nome do dataset cumulativo
    NUM_PROCESSOS = 16

    if not DESIRED_MATCHUPS:
        print("ERRO: A lista DESIRED_MATCHUPS está vazia.")
        return
    if NUM_PROCESSOS < 1:
        print("ERRO: NUM_PROCESSOS deve ser pelo menos 1.")
        return 

    print(f"Preparando {NUM_GAMES_ESTA_EXECUCAO} jogos para adicionar a '{FILENAME}'...")
    print(f"Modo: {MODE}")
    print(f"Matchups a incluir: {DESIRED_MATCHUPS}")

    start_time_prep = time.time()

    # Prepara lista de argumentos para as tarefas (passando strings de dificuldade)
    tasks_args = []
    num_matchups = len(DESIRED_MATCHUPS)
    games_per_matchup = NUM_GAMES_ESTA_EXECUCAO // num_matchups
    extra_games = NUM_GAMES_ESTA_EXECUCAO % num_matchups
    game_id_counter = 0

    for i, matchup_settings in enumerate(DESIRED_MATCHUPS):
        num_jogos_neste_matchup = games_per_matchup + (1 if i < extra_games else 0)
        if num_jogos_neste_matchup == 0: continue

        # Extrai as settings (strings de dificuldade) para P1 e P2
        p1_setting, p2_setting = None, None
        if MODE == 'mcts_vs_mcts':
            p1_setting, p2_setting = matchup_settings # Espera tuple ('diff1', 'diff2')
        elif MODE == 'mcts_vs_random':
            p1_setting = matchup_settings[0] # Espera ('diff1',) ou apenas 'diff1'
            p2_setting = None # Random não tem setting
        elif MODE == 'random_vs_mcts':
            p1_setting = None # Random não tem setting
            p2_setting = matchup_settings[0] # Espera ('diff2',) ou apenas 'diff2'

        # Validação básica das settings (não nulas quando esperado)
        if (MODE != 'random_vs_mcts' and p1_setting is None) or \
           (MODE != 'mcts_vs_random' and p2_setting is None and MODE != 'mcts_vs_mcts'):
             continue

        for _ in range(num_jogos_neste_matchup):
            game_id_counter += 1
            # Passa as strings de dificuldade (ou None) para o worker
            tasks_args.append((MODE, p1_setting, p2_setting, game_id_counter))

    if not tasks_args:
        print("Nenhuma tarefa de simulação foi criada.")
        return

    # Executa o Pool
    print("A iniciar Pool...")
    results = []
    try:
        with multiprocessing.Pool(processes=NUM_PROCESSOS) as pool:
            results = pool.map(simular_jogo_e_coletar_dados, tasks_args)
    except Exception as e:
        print(f"\nERRO durante execução do Pool: {e}")
        results = []

    # Contar vitórias
    # Contar vitórias por par de dificuldades
    win_counter = {}
    for winner, p1, p2 in results:
        key = (str(p1), str(p2))
        if key not in win_counter:
            win_counter[key] = {"PLAYER1": 0, "PLAYER2": 0, "Empate": 0}
        if winner == PLAYER1:
            win_counter[key]["PLAYER1"] += 1
        elif winner == PLAYER2:
            win_counter[key]["PLAYER2"] += 1
        else:
            win_counter[key]["Empate"] += 1

    print("\nResultados finais após todos os jogos:")
    for matchup, counts in win_counter.items():
        print(f"{matchup[0]} vs {matchup[1]} - P1: {counts['PLAYER1']} vitórias, P2: {counts['PLAYER2']} vitórias, Empates: {counts['Empate']}")

if __name__ == '__main__':
    main()